# **1: Download & Extract EuroSAT**

In [1]:

!pip install -q rasterio


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 34.2 MB/s eta 0:00:00


# 2: Download and Unzip EuroSAT AllBands

In [2]:
!wget -q --no-check-certificate https://madm.dfki.de/files/sentinel/EuroSATallBands.zip -O EuroSATallBands.zip
!unzip -q EuroSATallBands.zip -d EuroSAT


# 3: Convert .tif to .npy Using B11, B8, B4 + NDVI

In [3]:
import os
import numpy as np
import rasterio
from tqdm import tqdm

def convert_selected_bands_with_ndvi(
    src_root="EuroSAT/ds/images/remote_sensing/otherDatasets/sentinel_2/tif",
    dst_root="EuroSAT_npy_AllBands_NDVI"
):
    os.makedirs(dst_root, exist_ok=True)
    # All bands except B10 (index 9)
    selected_indices = [1,2,3,4,5,6,7,8,10,11,12]  # B1-B9, B11-B12

    for class_name in tqdm(os.listdir(src_root)):
        src_class_path = os.path.join(src_root, class_name)
        dst_class_path = os.path.join(dst_root, class_name)
        os.makedirs(dst_class_path, exist_ok=True)

        for fname in os.listdir(src_class_path):
            if not fname.endswith(".tif"):
                continue
            path = os.path.join(src_class_path, fname)
            out_path = os.path.join(dst_class_path, fname.replace(".tif", ".npy"))

            try:
                with rasterio.open(path) as src:
                    img = src.read()
                    img = np.clip(img, 0, 10000).astype(np.float32)

                    # Raw bands (scaled to 0‑1)
                    raw_bands = img[selected_indices] / 10000.0

                    # Indices
                    B3, B4, B8, B11 = img[2], img[3], img[7], img[11]   # note: indices w.r.t selected set
                    ndvi = (B8 - B4) / (B8 + B4 + 1e-6)
                    ndwi = (B3 - B8) / (B3 + B8 + 1e-6)
                    ndbi = (B11 - B8) / (B11 + B8 + 1e-6)
                    ndmi = (B8 - B11) / (B8 + B11 + 1e-6)
                    savi = (1.5 * (B8 - B4)) / (B8 + B4 + 0.5)

                    indices = np.stack([ndvi, ndwi, ndbi, ndmi, savi], axis=0)
                    indices = np.clip(indices, -1, 1)

                    final = np.concatenate([raw_bands, indices], axis=0).astype(np.float32)
                    np.save(out_path, final)

            except Exception as e:
                print(f" Failed: {fname} | {e}")

    print(f" NPY files saved in {dst_root}")

convert_selected_bands_with_ndvi()


100%|██████████| 10/10 [02:08<00:00, 12.87s/it]

 NPY files saved in EuroSAT_npy_AllBands_NDVI


# **4: Build Dataset Index + Compute Mean/Std**

In [4]:
import pandas as pd

def build_dataset_index(npy_root="EuroSAT_npy_AllBands_NDVI"):
    rows = []
    classes = sorted(os.listdir(npy_root))
    for cls in classes:
        for fname in os.listdir(os.path.join(npy_root, cls)):
            if fname.endswith(".npy"):
                rows.append({
                    "path": os.path.join(npy_root, cls, fname),
                    "label": cls
                })
    df = pd.DataFrame(rows)
    df['label_idx'] = df['label'].astype('category').cat.codes
    return df

df = build_dataset_index()

from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(df, stratify=df['label'], test_size=0.2, random_state=42)

def compute_mean_std(df):
    channel_sum = 0
    channel_squared_sum = 0
    total_pixels = 0

    for path in tqdm(df['path'], desc="Computing mean/std"):
        arr = np.load(path)
        c, h, w = arr.shape
        channel_sum += arr.sum(axis=(1, 2))
        channel_squared_sum += (arr ** 2).sum(axis=(1, 2))
        total_pixels += h * w

    mean = channel_sum / total_pixels
    std = np.sqrt(channel_squared_sum / total_pixels - mean ** 2)
    return mean.tolist(), std.tolist()

mean, std = compute_mean_std(df)
print("Mean:", mean)
print("Std:", std)


Computing mean/std: 100%|██████████| 27000/27000 [00:50<00:00, 531.24it/s]

Mean: [0.1117151603102684, 0.10418197512626648, 0.0946442037820816, 0.11991236358880997, 0.20029175281524658, 0.23738867044448853, 0.23010846972465515, 0.07321809232234955, 0.18205863237380981, 0.11181201785802841, 0.25996464490890503, 0.34858691692352295, -0.2825516164302826, -0.3988216817378998, 0.3988216817378998, 0.49681341648101807]
Std: [0.03314068540930748, 0.03930474445223808, 0.05918382853269577, 0.05657161399722099, 0.08597585558891296, 0.10856421291828156, 0.11165442317724228, 0.04037296772003174, 0.10013400763273239, 0.07593318074941635, 0.12302295863628387, 0.3300042450428009, 0.3387872874736786, 0.27285003662109375, 0.27285003662109375, 0.4637760818004608]


# 5: Define Transforms****

In [5]:
import torch.optim as optim

In [6]:
from torchvision import transforms

# Already computed from your dataset
#mean = [0.18205855758101852, 0.2301079463252315, 0.09464429615162037, 0.3485864438657407]
#std = [0.10013492998282264, 0.11165543609217363, 0.059183771408410114, 0.33000484745181274]

train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(60),
    transforms.Resize((64, 64)),
    #transforms.Normalize(mean=mean, std=std)
])

val_transform = transforms.Compose([
    transforms.Resize((64, 64)),
    #transforms.Normalize(mean=mean, std=std)
])


# 6: Dataset and DataLoader****

In [7]:
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
import torch
import numpy as np

train_df, val_df = train_test_split(df, stratify=df['label'], test_size=0.2, random_state=42)

class EuroSATDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.df = dataframe.reset_index(drop=True)
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        sample = np.load(row['path']).astype(np.float32)
        sample = torch.tensor(sample)

        # Calculate mean and std per sample
        mean = sample.mean(dim=[1, 2], keepdim=True)
        std = sample.std(dim=[1, 2], keepdim=True)

        # Normalize the sample
        sample = (sample - mean) / (std + 1e-5)  # Adding a small constant to avoid division by zero

        if self.transform:
            sample = self.transform(sample)

        label = torch.tensor(row['label_idx'], dtype=torch.long)
        return sample, label

train_ds = EuroSATDataset(train_df, transform=train_transform)
val_ds = EuroSATDataset(val_df, transform=val_transform)

train_loader = DataLoader(
    train_ds, batch_size=64, shuffle=True,
    num_workers=2, prefetch_factor=4, pin_memory=True
)

val_loader = DataLoader(
    val_ds, batch_size=64, shuffle=False,
    num_workers=2, prefetch_factor=4, pin_memory=True
)


**# 7: Define EfficientNet-B4 with 4 Channels**

In [8]:
# ...existing code...
from torchvision import models
import torch.nn as nn
import torch

class ResNet50_MultiChannel(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.model = models.resnet50(weights=None)
        # Modify first conv layer to accept 16 channels
        self.model.conv1 = nn.Conv2d(16, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.model.fc = nn.Linear(self.model.fc.in_features, num_classes)

    def forward(self, x):
        return self.model(x)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ResNet50_MultiChannel(num_classes=df['label_idx'].nunique())

if torch.cuda.device_count() > 1:
    print(f"✅ Using {torch.cuda.device_count()} GPUs via DataParallel")
    model = nn.DataParallel(model)

model = model.to(device)
# ...existing code...

**# 8: Loss + Optimizer**

In [9]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)


**# 9: Training + Validation Loops**

In [10]:
from sklearn.metrics import classification_report

def train_one_epoch(model, loader, optimizer, criterion, device, epoch, log_interval=10):
    model.train()
    total_loss, correct, total = 0, 0, 0
    batch_loss = 0

    for batch_idx, (x, y) in enumerate(loader, 1):
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        out = model(x)
        loss = criterion(out, y)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0) # Gradient clipping
        optimizer.step()

        total_loss += loss.item() * x.size(0)
        batch_loss += loss.item()
        correct += (out.argmax(1) == y).sum().item()
        total += y.size(0)

        if batch_idx % log_interval == 0:
            avg_loss = batch_loss / log_interval
            print(f"[Epoch {epoch}] Batch {batch_idx:04d} - Avg Loss: {avg_loss:.4f}")
            batch_loss = 0

    return total_loss / total, correct / total

def validate(model, loader, criterion, device, class_names):
    model.eval()
    total_loss, correct, total = 0, 0, 0
    y_true = []
    y_pred = []

    with torch.no_grad():
        for x, y in loader:
            x, y = x.to(device), y.to(device)
            out = model(x)
            loss = criterion(out, y)

            total_loss += loss.item() * x.size(0)
            correct += (out.argmax(1) == y).sum().item()
            total += y.size(0)

            y_true.extend(y.cpu().numpy())
            y_pred.extend(out.argmax(1).cpu().numpy())

    val_acc = correct / total
    print(f"\n Val Acc: {val_acc * 100:.2f}%\n")
    print(" Classification Report:")
    print(classification_report(y_true, y_pred, target_names=class_names))

    return total_loss / total, val_acc


# **10: Train the Model**

In [11]:
def run_training(train_loader, val_loader, df, num_epochs=10, save_path="./"):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = ResNet50_MultiChannel(num_classes=df['label_idx'].nunique())

    if torch.cuda.device_count() > 1:
        print(f"✅ Using {torch.cuda.device_count()} GPUs via DataParallel")
        model = nn.DataParallel(model)

    model = model.to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-5) # Added weight decay

    best_acc = 0.0
    class_names = sorted(df['label'].unique())

    for epoch in range(1, num_epochs + 1):
        train_loss, train_acc = train_one_epoch(model, train_loader, optimizer, criterion, device, epoch)
        val_loss, val_acc = validate(model, val_loader, criterion, device, class_names)

        print(f"\nTrain Loss: {train_loss:.4f}, Acc: {train_acc * 100:.2f}")
        print(f"Val Acc: {val_acc * 100:.2f}")

        if val_acc > best_acc:
            best_acc = val_acc
            model_path = f"{save_path}/best_model_epoch_{epoch:02d}.pth"
            torch.save(
    {
        "model_state": model.state_dict(),
        "class_to_idx": train_ds.df[["label", "label_idx"]]
                            .drop_duplicates()
                            .set_index("label_idx")["label"]
                            .to_dict()
    },
    model_path
)
            print(f" Saved new best model: {model_path}")

    print("Training complete.")


**Lights out and here we go**

In [12]:
run_training(train_loader, val_loader, df, num_epochs=20, save_path="./")


[Epoch 1] Batch 0010 - Avg Loss: 2.3583
[Epoch 1] Batch 0020 - Avg Loss: 2.2767
[Epoch 1] Batch 0030 - Avg Loss: 2.2861
[Epoch 1] Batch 0040 - Avg Loss: 2.2709
[Epoch 1] Batch 0050 - Avg Loss: 2.2841
[Epoch 1] Batch 0060 - Avg Loss: 2.2353
[Epoch 1] Batch 0070 - Avg Loss: 2.2420
[Epoch 1] Batch 0080 - Avg Loss: 2.2061
[Epoch 1] Batch 0090 - Avg Loss: 2.1816
[Epoch 1] Batch 0100 - Avg Loss: 2.1750
[Epoch 1] Batch 0110 - Avg Loss: 2.1469
[Epoch 1] Batch 0120 - Avg Loss: 2.1903
[Epoch 1] Batch 0130 - Avg Loss: 2.1153
[Epoch 1] Batch 0140 - Avg Loss: 2.0819
[Epoch 1] Batch 0150 - Avg Loss: 2.1036
[Epoch 1] Batch 0160 - Avg Loss: 2.0504
[Epoch 1] Batch 0170 - Avg Loss: 1.8860
[Epoch 1] Batch 0180 - Avg Loss: 1.8858
[Epoch 1] Batch 0190 - Avg Loss: 1.9002
[Epoch 1] Batch 0200 - Avg Loss: 1.8339
[Epoch 1] Batch 0210 - Avg Loss: 1.7457
[Epoch 1] Batch 0220 - Avg Loss: 1.6414
[Epoch 1] Batch 0230 - Avg Loss: 1.6655
[Epoch 1] Batch 0240 - Avg Loss: 1.6598
[Epoch 1] Batch 0250 - Avg Loss: 1.4721


**# Inference + Submission**

# **1. Load Best Model**

In [13]:
best_model_path = "/content/best_model_epoch_20.pth"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 1️⃣  Load the checkpoint
ckpt = torch.load(best_model_path, map_location=device)

# 2️⃣  Get the actual weight dict
state_dict = ckpt.get("state_dict", ckpt)          # Lightning-style or plain

# 3️⃣  Clean up the keys
clean_state_dict = {}
for k, v in state_dict.items():
    if k.startswith("module."): k = k[7:]          # drop 'module.'
    if k.startswith("model."):  k = k[6:]          # drop 'model.'
    clean_state_dict[k] = v

# 4️⃣  Build the model and load weights (non-strict so extras are ignored)
model = ResNet50_MultiChannel(num_classes=df["label_idx"].nunique())
missing, unexpected = model.load_state_dict(clean_state_dict, strict=False)

print("Missing keys :", missing)      # should be [] if arch is identical
print("Unexpected   :", unexpected)   # safely ignored

model.to(device).eval()

Missing keys : ['model.conv1.weight', 'model.bn1.weight', 'model.bn1.bias', 'model.bn1.running_mean', 'model.bn1.running_var', 'model.layer1.0.conv1.weight', 'model.layer1.0.bn1.weight', 'model.layer1.0.bn1.bias', 'model.layer1.0.bn1.running_mean', 'model.layer1.0.bn1.running_var', 'model.layer1.0.conv2.weight', 'model.layer1.0.bn2.weight', 'model.layer1.0.bn2.bias', 'model.layer1.0.bn2.running_mean', 'model.layer1.0.bn2.running_var', 'model.layer1.0.conv3.weight', 'model.layer1.0.bn3.weight', 'model.layer1.0.bn3.bias', 'model.layer1.0.bn3.running_mean', 'model.layer1.0.bn3.running_var', 'model.layer1.0.downsample.0.weight', 'model.layer1.0.downsample.1.weight', 'model.layer1.0.downsample.1.bias', 'model.layer1.0.downsample.1.running_mean', 'model.layer1.0.downsample.1.running_var', 'model.layer1.1.conv1.weight', 'model.layer1.1.bn1.weight', 'model.layer1.1.bn1.bias', 'model.layer1.1.bn1.running_mean', 'model.layer1.1.bn1.running_var', 'model.layer1.1.conv2.weight', 'model.layer1.1.bn2

ResNet50_MultiChannel(
  (model): ResNet(
    (conv1): Conv2d(16, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequentia

# **2. Build Class Index → Label Mapping**

In [14]:
!unzip -q testset.zip -d TestSet

In [15]:
##idx_to_class = {v: k for k, v in df[['label', 'label_idx']].drop_duplicates().set_index('label').to_dict()['label_idx'].items()}


# **3. Inference on Test NPY Files**

In [16]:
test_dir = "/content/TestSet/testset/testset"

In [17]:
import os
from glob import glob

print("Exists:", os.path.exists(test_dir))
print("Files found:", len(glob(os.path.join(test_dir, "*.npy"))))


Exists: True
Files found: 4232


In [18]:
#mean = torch.tensor([0.1821, 0.2301, 0.0946, 0.3486], device=device).view(4, 1, 1)
#std = torch.tensor([0.1001, 0.1117, 0.0592, 0.3300], device=device).view(4, 1, 1)


In [18]:
import os
import numpy as np
import torch
from tqdm import tqdm

test_dir = "/content/TestSet/testset/testset"
ckpt = torch.load("/content/best_model_epoch_20.pth", map_location=device)
model.load_state_dict(ckpt["model_state"])
idx_to_class = ckpt["class_to_idx"]      # <-- guaranteed to match training
model.eval()
test_files = sorted(glob(os.path.join(test_dir, "*.npy")))

#mean = torch.tensor([0.1821, 0.2301, 0.0946, 0.3486], device=device).view(4, 1, 1)
#std = torch.tensor([0.1001, 0.1117, 0.0592, 0.3300], device=device).view(4, 1, 1)

results = []

def process_test_data(path):
    x = np.load(path).astype(np.float32)
    x = np.transpose(x, (2, 0, 1))
    # No transpose needed, assuming the input is (C, H, W) after loading

    # Raw bands (scaled to 0-1)
    selected_indices = [1,2,3,4,5,6,7,9,10,11,8]   # B1-B9, B11-B12
    raw_bands = x[selected_indices] / 10000.0

    # Indices
    B3, B4, B8, B11 = x[2], x[3], x[7], x[11]   # note: indices w.r.t selected set
    ndvi = (B8 - B4) / (B8 + B4 + 1e-6)
    ndwi = (B3 - B8) / (B3 + B8 + 1e-6)
    ndbi = (B11 - B8) / (B11 + B8 + 1e-6)
    ndmi = (B8 - B11) / (B8 + B11 + 1e-6)
    savi = (1.5 * (B8 - B4)) / (B8 + B4 + 0.5)


    indices = np.stack([ndvi, ndwi, ndbi, ndmi, savi], axis=0)
    indices = np.clip(indices, -1, 1)

    x_final = np.concatenate([raw_bands, indices], axis=0).astype(np.float32)
    x_final = torch.tensor(x_final)

    #x_final = torch.nn.functional.interpolate(
     #       torch.tensor(x_final).unsqueeze(0),
      #      size=(64, 64),
       #     mode="bilinear",
        #    align_corners=False
         # ).squeeze(0)

    # Calculate mean and std per sample
    mean = x_final.mean(dim=[1, 2], keepdim=True)
    std = x_final.std(dim=[1, 2], keepdim=True)

    # Normalize the sample
    x_final = (x_final - mean) / (std + 1e-5)  # Adding a small constant to avoid division by zero

    return x_final


# Update inference loop
with torch.no_grad():
    for path in tqdm(test_files, desc="Running Inference"):
        x_final = process_test_data(path)
        x_final = x_final.to(device, dtype=torch.float32)
        x_final = x_final.unsqueeze(0)

        out = model(x_final)
        pred = out.argmax(1).item()
        label = idx_to_class[pred]
        results.append((os.path.basename(path).replace(".npy", ""), label))


Running Inference: 100%|██████████| 4232/4232 [00:42<00:00, 100.18it/s]


# **4. Generate Submission File**

In [19]:
import pandas as pd

# 'results' contains tuples like: [('test_0', 'Forest'), ('test_1', 'River'), ...]

# Convert to DataFrame with correct format
submission_df = pd.DataFrame(results, columns=["test_id", "label"])

# Extract numeric part of test_id and convert to int
submission_df["test_id"] = submission_df["test_id"].str.extract(r"(\d+)").astype(int)

# Sort by test_id to match sample_submission.csv order
submission_df = submission_df.sort_values("test_id").reset_index(drop=True)

# Save to CSV
submission_df.to_csv("submission1105_20.csv", index=False)


**Submission**

In [20]:
from collections import Counter

# Extract just the predicted labels
predicted_labels = [label for _, label in results]

# Count occurrences of each class
class_counts = Counter(predicted_labels)

# Get all possible classes (in case some are missing from predictions)
all_classes = sorted(idx_to_class.values())

print("\nPredicted class distribution:")
for class_name in all_classes:
    count = class_counts.get(class_name, 0)
    print(f"{class_name}: {count}")



Predicted class distribution:
AnnualCrop: 490
Forest: 280
HerbaceousVegetation: 338
Highway: 392
Industrial: 174
Pasture: 799
PermanentCrop: 464
Residential: 737
River: 103
SeaLake: 455
